In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import datetime

In [2]:
df = pd.DataFrame({
   'id':[],
   'name':[],
   'url':[],
})

In [3]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 YaBrowser/23.11.0.0 Safari/537.36'
      }

In [4]:
def zakup_df (soup, data = df): 
    list_ = soup.find_all('div', {'class': 'col-8 pr-0'})
    for i in list_:
        quote = {}     
        temp = i.find('div', attrs = {'registry-entry__header-mid__number'}).find('a').text
        quote['id'] = re.findall("\d+", temp)[0]
        try:
               quote['name'] = i.find('div', attrs = {'registry-entry__body-value'}).text 
        except Exception:
            quote['name'] = 'Нет названия'
        quote['url'] = 'https://zakupki.gov.ru/' + i.find('div', attrs = {'registry-entry__header-mid__number'}).find('a').get('href')
        df.loc[len(df)] = quote

In [5]:
for i in range(1, 960): #На сайте 50 страниц (1000 книг по 20 на странице)
    url = 'https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&search-filter=Дате+размещения&pageNumber=' + str(i) + '&sortDirection=false&recordsPerPage=_50&showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&currencyIdGeneral=-1&publishDateFrom=01.01.2023&publishDateTo=27.01.2024&okpd2IdsWithNested=on&okpd2Ids=8873937%2C8873938&okpd2IdsCodes=62%2C63'
    r = requests.get(url, headers = headers, timeout = 10)
    soup = BeautifulSoup(r.content, 'html.parser')    
    zakup_df(soup, df)  
    

18 минут 20 сек парсилось

In [7]:
import datetime
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"file_{current_time}.csv"
df.to_csv(filename)

In [10]:
data = pd.read_csv('./file_2024-01-28_22-24-03.csv', encoding='utf-8')
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.head(3)

,id,name,url
0,122200001024000004,Оказание информационных услуг с использованием...,https://zakupki.gov.ru//epz/order/notice/zk20/...
1,822200000224000005,Поставка электронного периодического справочника,https://zakupki.gov.ru//epz/order/notice/zk20/...
2,32413220504,Оказание услуг по технической и контентной под...,https://zakupki.gov.ru//epz/order/notice/notic...


In [11]:
data.shape

(47950, 3)